In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

# Parámetros
image_size = (64, 64)  # Puedes usar 64x64 o más si tu GPU lo permite
batch_size = 16

# Generador de datos
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    '../Data/raw',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    '../Data/raw',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Modelo CNN
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(4, activation='softmax')  # 4 clases
])

# Compilar
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Entrenar
model.fit(train_generator, epochs=10, validation_data=val_generator)


Found 105 images belonging to 4 classes.
Found 25 images belonging to 4 classes.


c:\Users\34666\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\34666\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 279ms/step - accuracy: 0.5127 - loss: 1.5524 - val_accuracy: 0.8400 - val_loss: 1.0992
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 167ms/step - accuracy: 0.5803 - loss: 1.0593 - val_accuracy: 0.5600 - val_loss: 0.8825
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 163ms/step - accuracy: 0.7165 - loss: 0.8165 - val_accuracy: 0.6400 - val_loss: 0.7053
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 172ms/step - accuracy: 0.7423 - loss: 0.6068 - val_accuracy: 0.8400 - val_loss: 0.5354
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 167ms/step - accuracy: 0.7918 - loss: 0.5164 - val_accuracy: 0.8400 - val_loss: 0.4730
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 153ms/step - accuracy: 0.8022 - loss: 0.3921 - val_accuracy: 0.8000 - val_loss: 0.4079
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 166ms/step - accuracy: 0.8864 - loss: 0.3805 - val_accuracy: 0.8800 - val_loss: 0.3396
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 157ms/step - accuracy: 0.8858 - loss: 0.3528 - val_accuracy: 0.8000 - val_loss:

In [7]:
clases = ['despejado', 'muy_nublado', 'nublado', 'otros']  # Asegúrate de que coincidan con las carpetas


In [9]:
import os
from tensorflow.keras.preprocessing import image
import numpy as np

# Ruta a la carpeta con imágenes nuevas
carpeta_predicciones = '../Data/raw/Predict'

# Lista de imágenes
imagenes = [f for f in os.listdir(carpeta_predicciones) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

for nombre in imagenes:
    ruta = os.path.join(carpeta_predicciones, nombre)

    # Cargar y preparar la imagen
    img = image.load_img(ruta, target_size=(64, 64))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Predecir
    pred = model.predict(img_array)
    clase = clases[np.argmax(pred)]
    confianza = np.max(pred)

    print(f'{nombre}: {clase} ({confianza:.2f})')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1a.PNG: nublado (1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1aa.PNG: despejado (0.97)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1b.PNG: muy_nublado (0.59)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1c.PNG: nublado (0.49)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1d.PNG: despejado (0.96)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1e.PNG: nublado (0.97)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1f.PNG: nublado (0.50)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1g.PNG: muy_nublado (0.84)


In [10]:
model.save("modelo_meteorologico.h5")
